<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=39352795d5fd9d5469ef79cebb55cf31d7ae96aa73303e6771b0622e4eddcce9
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 11:01:12
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: -1.31 L (-0.94%)
Current PnL: -22.36 L (-15.04%)
CY Booked + Current PnL: -10.92 L (-7.35%)
-------------------
Total profit:  93.65 K
Total loss:  -23.30 L
-------------------
Total Booked + Current PnL: 16.06 L (13.19%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.34%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.23 L (64.29%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.29%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,10.86,51.0,H-MC,3.86,191330.0,26660.0,16665.0,-0.73,16.19,8.71,26.31,89.0,1.60,1.39,25.46,X40N,NTT,AC
77,TTKPRESTIG,770.00,95.09,38.0,M-SC,2.78,84030.0,-16747.0,16840.0,-1.80,-16.62,20.04,0.09,245.0,-0.99,0.61,10.40,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.92,107060.0,6556.0,22547.0,-1.76,6.52,21.06,28.96,79.0,0.29,0.78,42.71,MH,ATH,METALS
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196627.0,-3511.0,24401.0,0.50,-1.75,12.41,10.44,4.0,-0.14,1.43,3.71,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.41,42.0,H-LC,4.06,214204.0,3838.0,24955.0,0.05,1.82,11.65,13.68,37.0,0.15,1.56,18.65,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.00,-22.16,44.0,M-LC,2.36,246114.0,-30409.0,140482.0,2.06,-11.00,57.08,39.81,54.0,-0.22,1.79,17.95,XY24,NTT,AUTO
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.90,83010.0,-19200.0,112586.0,1.42,-18.78,135.63,91.37,208.0,-0.17,0.61,34.75,XY25,NTT,FINANCE
34,ICICIGI,2260.25,-18.25,54.0,H-MC,1.87,140548.0,4415.0,26704.0,0.93,3.24,19.00,22.86,91.0,0.17,1.02,16.27,X40,ATH,INSURANCE
64,SHALBY,327.00,1188.68,66.0,M-SC,12.77,175260.0,-5883.0,51351.0,0.76,-3.25,29.30,25.10,235.0,-0.11,1.28,40.45,XY24,NTT,HEALTHCARE
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196627.0,-3511.0,24401.0,0.50,-1.75,12.41,10.44,4.0,-0.14,1.43,3.71,X40,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HAPPSTMNDS,1488.71,-20.51,27.0,H-SC,10.53,85848.0,-42317.0,152346.0,-2.98,-33.02,177.46,85.85,132.0,-0.28,0.63,0.00,AR,ATH,IT
46,KPIGREEN,731.05,-3.26,31.0,H-SC,8.76,115088.0,-10209.0,69133.0,-2.94,-8.15,60.07,47.03,141.0,-0.15,0.84,44.10,MH,ATH,POWER
32,HINDZINC,730.22,21.48,55.0,M-LC,2.80,197968.0,-7108.0,119672.0,-2.50,-3.47,60.45,54.89,52.0,-0.06,1.44,19.79,X5K,ATH,METALS
54,RAJESHEXPO,518.00,1691.67,41.0,L-SC,2.60,48895.0,-88282.0,88373.0,-2.39,-64.36,180.74,0.07,267.0,-1.00,0.36,21.60,OX40N,NTT,JEWELLERY
39,INDUSINDBK,1800.00,236.83,37.0,L-MC,12.04,41322.0,-35884.0,61276.0,-2.13,-46.48,148.29,32.89,258.0,-0.59,0.30,13.80,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,91.37,49.0,H-SC,10.19,121203.0,-21276.0,40797.0,-0.18,-14.93,33.66,13.70,163.0,-0.52,0.88,43.05,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,95.09,38.0,M-SC,2.78,84030.0,-16747.0,16840.0,-1.80,-16.62,20.04,0.09,245.0,-0.99,0.61,10.40,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,48.0,H-SC,1.40,219591.0,-50076.0,86409.0,-0.51,-18.57,39.35,13.47,138.0,-0.58,1.60,11.07,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,34.0,H-SC,1.85,140691.0,-35212.0,77239.0,-0.18,-20.02,54.90,23.89,149.0,-0.46,1.03,21.51,OX40N,NTT,CERAMICS
66,SIS,528.0,2081.35,40.0,H-SC,4.23,88039.0,-22993.0,46599.0,-0.25,-20.71,52.93,21.26,156.0,-0.49,0.64,18.95,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,58.0,M-MC,10.28,230180.0,5218.0,162438.0,-2.09,2.32,70.57,74.53,192.0,0.03,1.68,35.32,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,10.86,51.0,H-MC,3.86,191330.0,26660.0,16665.0,-0.73,16.19,8.71,26.31,89.0,1.60,1.39,25.46,X40N,NTT,AC
84,VOLTAS,1530.00,-3.32,41.0,H-MC,3.04,202935.0,11193.0,26564.0,-1.42,5.84,13.09,19.69,99.0,0.42,1.48,13.03,XY25,NTT,AC
17,CAMS,5211.76,-5.61,44.0,H-SC,1.76,107243.0,5239.0,38683.0,-0.99,5.14,36.07,43.06,122.0,0.14,0.78,23.13,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,53.0,M-LC,6.83,154837.0,13997.0,30487.0,-1.87,9.94,19.69,31.59,66.0,0.46,1.13,39.06,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.92,107060.0,6556.0,22547.0,-1.76,6.52,21.06,28.96,79.0,0.29,0.78,42.71,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-44.45,33.0,M-SC,1.50,93960.0,2462.0,76286.0,0.30,2.69,81.19,86.07,223.0,0.03,0.68,36.45,XR,NTT,DURABLES
11,BANKINDIA,190.00,-29.19,56.0,H-MC,8.23,182268.0,2460.0,109944.0,-1.04,1.37,60.32,62.52,88.0,0.02,1.33,35.92,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.23,125218.0,1882.0,131880.0,-0.46,1.53,105.32,108.45,119.0,0.01,0.91,24.22,AR,ATH,MISC
86,WIPRO,420.00,-16.06,33.0,M-LC,5.64,147219.0,-3726.0,113182.0,-1.88,-2.47,76.88,72.51,53.0,-0.03,1.07,3.41,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-17.64,29.0,H-LC,7.04,207953.0,-43815.0,97031.0,-1.70,-17.40,46.66,21.13,27.0,-0.45,1.52,11.86,X40,ATH,PAINTS
44,JIOFIN,387.00,-5.28,30.0,H-LC,12.97,257398.0,-9967.0,75418.0,-0.76,-3.73,29.30,24.48,48.0,-0.13,1.88,49.19,XY24,BTT,FINANCE
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274602.0,-71354.0,146226.0,-1.22,-20.63,53.25,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.19,31.0,H-LC,8.17,287260.0,-28582.0,141246.0,-0.35,-9.05,49.17,35.67,5.0,-0.20,2.09,3.55,X40N,ATH,FMCG
30,HCLTECH,1943.91,-3.66,36.0,H-LC,9.92,220491.0,-21433.0,84713.0,-1.36,-8.86,38.42,26.15,8.0,-0.25,1.61,4.19,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.23,41.0,H-LC,1.68,230242.0,-15909.0,95113.0,-1.87,-6.46,41.31,32.18,16.0,-0.17,1.68,27.25,X200,ATH,IT
42,ITC,452.0,-39.19,41.0,H-LC,2.18,196627.0,-3511.0,24401.0,0.50,-1.75,12.41,10.44,4.0,-0.14,1.43,3.71,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.83,43.0,H-LC,2.62,156175.0,-29920.0,77400.0,-1.04,-16.08,49.56,25.51,15.0,-0.39,1.14,15.09,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,42.0,H-LC,4.06,214204.0,3838.0,24955.0,0.05,1.82,11.65,13.68,37.0,0.15,1.56,18.65,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.41,43.0,H-LC,4.52,272610.0,7184.0,49615.0,-0.72,2.71,18.20,21.40,11.0,0.14,1.99,9.99,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274602.0,-71354.0,146226.0,-1.22,-20.63,53.25,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.23,44.0,H-LC,7.16,224942.0,-9202.0,57833.0,-0.74,-3.93,25.71,20.77,86.0,-0.16,1.64,2.72,X40N,NTT,BREWERIES
83,VBL,671.64,-20.19,31.0,H-LC,8.17,287260.0,-28582.0,141246.0,-0.35,-9.05,49.17,35.67,5.0,-0.20,2.09,3.55,X40N,ATH,FMCG
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196627.0,-3511.0,24401.0,0.50,-1.75,12.41,10.44,4.0,-0.14,1.43,3.71,X40,NTT,FMCG
30,HCLTECH,1943.91,-3.66,36.0,H-LC,9.92,220491.0,-21433.0,84713.0,-1.36,-8.86,38.42,26.15,8.0,-0.25,1.61,4.19,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.83,43.0,H-LC,2.62,156175.0,-29920.0,77400.0,-1.04,-16.08,49.56,25.51,15.0,-0.39,1.14,15.09,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196627.0,-3511.0,24401.0,0.50,-1.75,12.41,10.44,4.0,-0.14,1.43,3.71,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.54,204705.0,205.0,41207.0,-1.03,0.10,20.13,20.25,10.0,0.00,1.49,8.84,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.64,29.0,H-LC,7.04,207953.0,-43815.0,97031.0,-1.70,-17.40,46.66,21.13,27.0,-0.45,1.52,11.86,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,42.0,H-LC,4.06,214204.0,3838.0,24955.0,0.05,1.82,11.65,13.68,37.0,0.15,1.56,18.65,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274602.0,-71354.0,146226.0,-1.22,-20.63,53.25,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-19.96,42.0,H-LC,8.12,311832.0,-1056.0,172755.0,-1.40,-0.34,55.40,54.87,3.0,-0.01,2.27,6.29,X40,BTT,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.18,196627.0,-3511.0,24401.0,0.50,-1.75,12.41,10.44,4.0,-0.14,1.43,3.71,X40,NTT,FMCG
83,VBL,671.64,-20.19,31.0,H-LC,8.17,287260.0,-28582.0,141246.0,-0.35,-9.05,49.17,35.67,5.0,-0.20,2.09,3.55,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,42.0,H-LC,7.98,247296.0,-14323.0,133540.0,-0.94,-5.47,54.00,45.57,7.0,-0.11,1.80,5.23,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,41.46,80352.0,-33197.0,73249.0,-1.23,-29.24,91.16,35.27,268.0,-0.45,0.59,96.66,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.90,83010.0,-19200.0,112586.0,1.42,-18.78,135.63,91.37,208.0,-0.17,0.61,34.75,XY25,NTT,FINANCE
49,MASFIN,398.61,-19.15,41.0,H-SC,12.53,91890.0,-6090.0,27696.0,-0.81,-6.22,30.14,22.05,152.0,-0.22,0.67,33.57,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.15,48.0,H-MC,10.92,107060.0,6556.0,22547.0,-1.76,6.52,21.06,28.96,79.0,0.29,0.78,42.71,MH,ATH,METALS
78,UJJIVANSFB,60.00,91.37,49.0,H-SC,10.19,121203.0,-21276.0,40797.0,-0.18,-14.93,33.66,13.70,163.0,-0.52,0.88,43.05,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1188.68,66.0,M-SC,12.77,175260.0,-5883.0,51351.0,0.76,-3.25,29.30,25.10,235.0,-0.11,1.28,40.45,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.90,83010.0,-19200.0,112586.0,1.42,-18.78,135.63,91.37,208.0,-0.17,0.61,34.75,XY25,NTT,FINANCE
71,TANLA,1963.11,-18.69,59.0,H-SC,16.88,218499.0,-52262.0,382220.0,-0.92,-19.30,174.93,121.86,133.0,-0.14,1.59,70.93,AR,ATH,IT
81,VAIBHAVGBL,521.00,54.82,53.0,H-SC,5.61,134412.0,-48363.0,159426.0,-1.12,-26.46,118.61,60.77,125.0,-0.30,0.98,20.39,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3613.51,50.0,M-SC,7.14,115805.0,-31563.0,83380.0,-0.20,-21.42,72.00,35.16,236.0,-0.38,0.84,18.28,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.25
1,25,43.71
2,50,74.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.93
LC    32.41
MC    23.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.34
X40      14.74
X40N     11.39
XR       11.16
XY25     10.15
AR        8.33
OX40N     7.90
X200      1.68
MH        1.62
X5K       1.44
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.39
H-LC    25.93
H-MC    20.67
M-SC    13.06
M-LC     5.43
M-MC     2.66
L-SC     1.48
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.02,-6.67,41.88
IT,12.85,-19.20,84.20
FINANCE,10.08,-18.76,70.58
MISC,6.99,-19.70,86.25
BANKS,6.16,-18.05,80.15
PAINTS,5.66,-18.97,36.71
ELECTRICAL,5.46,-9.78,48.30
INSURANCE,3.77,-6.87,46.81
AC,3.58,1.44,15.06


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3218684.0,22
XR,1378436.0,14
AR,1189448.0,9
X40,824462.0,10
X40N,641052.0,9
OX40N,562650.0,10
XY25,447815.0,7
SR,253990.0,2
X5K,119672.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3763375.0,29
M-SC,1451844.0,17
H-MC,1268234.0,15
H-LC,1262254.0,15
M-LC,403823.0,4
M-MC,313152.0,2
L-SC,259315.0,3
L-MC,61276.0,1
L-LC,39729.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253262.0      6
M-SC       XY24       830520.0      7
H-SC       AR         808591.0      5
           XR         799011.0      7
H-MC       XY24       575363.0      4
H-LC       X40        525126.0      5
H-SC       X40N       316011.0      4
M-MC       XY24       313152.0      2
H-SC       OX40N      263377.0      4
           SR         253990.0      2
H-LC       X40N       240286.0      3
H-MC       X40        223203.0      4
H-LC       AR         210940.0      2
H-MC       XY25       180129.0      2
M-LC       XY24       170969.0      2
L-SC       XR         170942.0      2
M-SC       AR         169917.0      2
           OX40N      138607.0      4
           XR         124081.0      2
M-LC       X5K        119672.0      1
H-LC       XY25       115371.0      3
M-LC       XR         113182.0      1
M-SC       XY25       112586.0      1
H-MC       XR         109944.0      1
H-LC       X200        95113.0      1
L-SC       OX40N       88373.0      1
H-MC       X40N        84755.0      2
M-SC       X40         76133.0      1
H-LC       XY24        75418.0      1
H-MC       OX40N       72293.0      1
H-SC       MH          69133.0      1
L-MC       XR          61276.0      1
L-LC       XY25        39729.0      1
H-MC       MH          22547.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
